In [2]:
import tushare as ts
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


In [3]:
pro = ts.pro_api()


In [4]:
# https://tushare.pro/document/2?doc_id=25
# get Constituents of SSE
data = pro.stock_basic(exchange='SSE', list_status='L',
                       fields='ts_code,symbol,name,area,industry,list_date')
# print(data.head())
# print(len(data))

# https://tushare.pro/document/2?doc_id=27
# the histories data of a stock
stock_data = pro.daily(ts_code='600000.SH',
                       start_date='20020901', end_date='20220901')
print(stock_data.head())
print(len(stock_data))


     ts_code trade_date  open  high   low  close  pre_close  change  pct_chg  \
0  600000.SH   20220901  7.23  7.29  7.20   7.23       7.27   -0.04  -0.5502   
1  600000.SH   20220831  7.17  7.28  7.15   7.27       7.19    0.08   1.1127   
2  600000.SH   20220830  7.14  7.20  7.13   7.19       7.15    0.04   0.5594   
3  600000.SH   20220829  7.16  7.21  7.12   7.15       7.22   -0.07  -0.9695   
4  600000.SH   20220826  7.23  7.26  7.18   7.22       7.20    0.02   0.2778   

         vol      amount  
0  232983.88  169134.999  
1  395661.30  286509.309  
2  225419.28  161534.933  
3  276003.24  197253.687  
4  175762.35  126876.441  
4747


In [16]:
pre_stock_data = pro.daily(
    ts_code='600000.SH', start_date='20020830', end_date='20220831')
print(pre_stock_data.head())

     ts_code trade_date  open  high   low  close  pre_close  change  pct_chg  \
0  600000.SH   20220831  7.17  7.28  7.15   7.27       7.19    0.08   1.1127   
1  600000.SH   20220830  7.14  7.20  7.13   7.19       7.15    0.04   0.5594   
2  600000.SH   20220829  7.16  7.21  7.12   7.15       7.22   -0.07  -0.9695   
3  600000.SH   20220826  7.23  7.26  7.18   7.22       7.20    0.02   0.2778   
4  600000.SH   20220825  7.16  7.22  7.15   7.20       7.15    0.05   0.6993   

         vol      amount  
0  395661.30  286509.309  
1  225419.28  161534.933  
2  276003.24  197253.687  
3  175762.35  126876.441  
4  157725.56  113287.117  


In [5]:
# 予測変数:is_up
stock_data.groupby("change").size()
stock_data["is_up"] = stock_data["change"].map(lambda x: 1 if x > 0 else 0)
stock_data.groupby("is_up").size()

is_up
0    2449
1    2298
dtype: int64

In [6]:
# TODO: end_date 変数
# https://note.nkmk.me/python-pandas-datetime-timestamp/
pre_stock_data = pro.daily(
    ts_code='600000.SH', start_date='20020830', end_date='20220831') 

stock_data["pre_open"] = pre_stock_data["open"]
stock_data["pre_high"] = pre_stock_data["high"]
stock_data["pre_low"] = pre_stock_data["low"]
stock_data["pre_vol"] = pre_stock_data["vol"]
stock_data["pre_amount"] = pre_stock_data["amount"]

# stock_data.info()

In [21]:
# Model: Logistic regression analysis
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

# 説明変数: T-1 (open high low close amount)
# TODO: T-1 ? T-x
# 10 days (https://www.google.com/search?q=python+running+average) SMA
X = stock_data[["pre_open", "pre_high", "pre_low", "pre_close", "pre_vol", "pre_amount"]]

Y = stock_data["is_up"]
X_train, X_test, y_train, y_test = train_test_split(X, Y, random_state=0)

# TODO:random forest / SVM
model = LogisticRegression()
clf = model.fit(X_train, y_train)

print("train result:", clf.score(X_train, y_train))
print("test result:", clf.score(X_test, y_test))


train result: 0.5303370786516854
test result: 0.4835720303285594


In [22]:
from sklearn.preprocessing import StandardScaler

sc = StandardScaler()
sc.fit(X_train)
X_train_std = sc.transform(X_train)
X_test_std = sc.transform(X_test)

clf = model.fit(X_train_std, y_train)
print("train:", clf.score(X_train_std, y_train))
print("test:", clf.score(X_test_std, y_test))


train: 0.5297752808988764
test: 0.4877843302443134


In [9]:
# 偏回帰係数
clf.coef_
# 0.5303701 is pre_high

array([[-0.10859816,  0.74862693,  0.04998734, -0.63189191,  0.01721982,
        -0.0013053 ]])

In [10]:
# Calculate the exponential of all elements in the input array.
np.exp(clf.coef_)


array([[0.89709083, 2.11409521, 1.05125779, 0.53158514, 1.01736894,
        0.99869555]])

In [11]:
# 株価予測の手法を調査して、使えそうなものをまとめる
# https://atrae.docbase.io/posts/2563322
# https://www.kaggle.com/search?q=stock+forecasting
